## Web Search Test
The **Web Search Test** ant demonstrates the use of a Web API secured using an API key with an auth header. This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

## First steps
- First, download AntRunLib from Nuget
- At least once, set up the environment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.7.1"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAiConfigFactory.Get();
var client = GetOpenAiClient(config);


Installed Packages AntRunnerLib, 0.7.1

## Ensure the Web Search Test assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search Test" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** environment variable.

"Web Search Test" contains the following files:
```
│   instructions.md - The instructions
│   manifest.json - The base assistant definition
│
└───OpenAPI
        api.bing.microsoft.com.json - A swagger file defining a tool named 'crawl' that uses web search
        auth.json - See below... 
```        

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case, a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```json
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
`AssistantUtility.Create` acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search Test", config);
Console.WriteLine(assistantId)

asst_DoYa5w2Kegl2anLzDE23ZZCd


## Run the Assistant

`output.Dialog` shows the conversation.

In [3]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search Test",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer. You can perform multiple searches if necessary",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
Console.WriteLine(output.Dialog)


User: Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer. You can perform multiple searches if necessary

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.today()

# Calculate the upcoming weekend dates
# Assuming the weekend is Saturday and Sunday
days_ahead = 5 - today.weekday()  # Calculate how many days until Saturday
if days_ahead < 0:  # If today is after Saturday (i.e., Sunday), calculate days until next Saturday
    days_ahead += 7

saturday = today + timedelta(days=days_ahead)
sunday = saturday + timedelta(days=1)

today_date_str = today.strftime("%Y-%m-%d")
saturday_date_str = saturday.strftime("%Y-%m-%d")
sunday_date_str = sunday.strftime("%Y-%m-%d")

today_date_str, saturday_date_str, sunday_date_str
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on September 21, 

In [4]:
Console.WriteLine(output.LastMessage)

Here are some things to do in Atlanta this weekend (September 21-22, 2024):

### Saturday, September 21, 2024
1. **Adair Park Festival**: A charming neighborhood festival with plenty of pie, a beer garden, artist market, kid activities, and more. Admission is free, but a pie-tasting pass costs $10.
   - [More details on Atlanta on the Cheap](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

2. **Chamblee City Hall FREE Event**: Join the community for various activities and entertainment.
   - [More details on Atlanta on the Cheap](https://www.atlantaonthecheap.com/fall-festivals-atlanta/)

3. **The Wallflowers Concert**: Enjoy live music by The Wallflowers at Frederick Brown Jr Amphitheatre.
   - [More details on Atlanta Events](https://atlanta-ga.events/september/)

4. **Men Are From Mars, Women Are From Venus**: A humorous show at Gas South Theater.
   - [More details on Atlanta Events](https://atlanta-ga.events/september/)

5. **Atlanta Greek Festival**: Held at the Greek 

## Clean Up

In [5]:
var assistant = (await client.AssistantList())?.Data?.FirstOrDefault(o => o.Name == "Web Search Test");
if(assistant != null) {
    await client.AssistantDelete(assistant.Id);
    Console.WriteLine("Deleted assistant");
}
else
{
    Console.WriteLine("Didn't find MsGraphUserProfile");
}

Deleted assistant
